In [ ]:
import os , argparse
import src.utils.setup as setup
from train import train
import tensorflow as tf
from decord import VideoLoader, VideoReader , cpu , gpu
from decord.bridge import set_bridge
import numpy as np

parser = argparse.ArgumentParser(description='zulowfat')
parser.add_argument('--experiment', 
                    type=str, 
                    default='cfgs/kinetics400/slowfast.yml', 
                    help='relative path to the experiment .yml')
ARGS = parser.parse_args(args=[])
cfg = setup.init(ARGS)

In [ ]:
from src.data.dataloader import InputReader
from src.data.utils import get_frames_idx

IR= InputReader(cfg,'train')
dstxt=IR.dstxt

line = next(iter(dstxt))
print(line)

for step, line in enumerate(dstxt):
    print(line)
    if step == 2: break

In [ ]:
get_frames_idx(100,32,2)

In [ ]:
import os , time
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
import tensorflow as tf
import matplotlib.pyplot as plt

def random_short_side_resize(clips, min_size, max_size):
    size = tf.random.uniform([], min_size, max_size, tf.float32)
    num_views = tf.shape(clips)[0]

    height = tf.cast(tf.shape(clips)[2], tf.float32)
    width = tf.cast(tf.shape(clips)[3], tf.float32)
    
    if (width <= height and width == size) or (
        height <= width and height == size):
        return clips
    new_width = size
    new_height = size
    if width < height:
      new_height = tf.math.floor((height / width) * size)
    else:
      new_width = tf.math.floor((width / height) * size)

    new_height = tf.cast(new_height, tf.int32)
    new_width = tf.cast(new_width, tf.int32)

    frames = [tf.image.resize(clips[i], [new_height, new_width])for i in range(num_views)]
    frames = tf.stack(frames, 0)
    frames.set_shape([None, *frames.shape[1:]])

    return tf.cast(frames, clips.dtype)

def normalize(clips, mean, std, norm_value=255):
    clips = tf.cast(clips, tf.float32)
    mean = tf.cast(mean, tf.float32)
    std = tf.cast(std, tf.float32)
    
    ## bs * nclips * crop * crop * 3
    shapes = tf.shape(clips)
    all_frames = tf.reshape(clips, [-1, shapes[-3], shapes[-2], shapes[-1]])
    ## nclips * crop * crop * 3
    all_frames /= norm_value
    
    def _normalize(frame, mean, std):
        frame = frame - mean
        return frame / std
    
    all_frames = tf.vectorized_map(
        lambda x: _normalize(x, mean, std),
        all_frames
    )
    print("Min and max pixel values:", all_frames.numpy().min(), all_frames.numpy().max())

    ## bs * nclips * crop * crop * 3
    return tf.reshape(all_frames, tf.shape(clips))

x = tf.random.uniform((1, 256, 320, 270, 3),maxval=255,dtype=tf.int32)
plt.imshow(tf.squeeze(x)[0])

#res_frames = random_short_side_resize(x,256,320)

In [ ]:
t = time.time()
frames = normalize(x , [0.45, 0.45, 0.45] , [0.225, 0.225, 0.225])
plt.imshow(tf.squeeze(frames)[0])
tt = time.time()
print(str(tt-t))

In [ ]:
p = 0.5
r = tf.random.uniform([],maxval=1)
if r > p: print(r.numpy())
else: print(r.numpy())